# Spectrum

In [1]:
import pandas as pd
import astropy.units as u
import numpy as np
import re
import matplotlib.pyplot as plt
from astroquery.simbad import Simbad
from astropy.coordinates import SkyCoord
from pandas.plotting import scatter_matrix

In [2]:
catalog = pd.read_csv("markeddata/gevtevsimbad_TeVspectrum.txt", sep='\t', index_col=0)

In [3]:
catalog.head()

,level_0,gev_0FGL_Name,gev_1FGL_Name,gev_1FHL_Name,gev_2FGL_Name,gev_ASSOC1,gev_ASSOC2,gev_ASSOC_GAM1,gev_ASSOC_GAM2,gev_ASSOC_GAM3,...,tev_tevcat_name,tev_tgevcat_id,tev_tgevcat_name,tev_where,marked,tev_0.3TeV,tev_1TeV,tev_3TeV,tev_10TeV,tev_30TeV
0,83,b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',...,b'TeV J1745-290',106,b'TeV J1745-2900',b'gal',0,3.341668e-11,2.440195e-12,1.828992e-13,5.610705e-15,6.128221e-17
1,92,b' ',b' ',b' ',b' ',b'HESS J1841-055 ',b' ',b' ',b' ',b' ',...,b'TeV J1840-055',125,b'TeV J1840-0533',b'gal',0,6.310589e-11,1.207092e-11,8.365091e-13,4.356098e-14,2.865129e-15
2,83,b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',...,b'TeV J1745-290',106,b'TeV J1745-2900',b'gal',0,3.341668e-11,2.440195e-12,1.828992e-13,5.610705e-15,6.128221e-17
3,86,b' ',b'1FGL J1746.4-2849c',b'1FHL J1746.3-2851 ',b'2FGL J1746.6-2851c',b'PWN G0.13-0.11 ',b' ',b' ',b'3EG J1746-2851',b' ',...,b'TeV J1746-289',-9223372036854775808,b'',b'gal',0,1.647138e-11,8.320850e-14,2.778103e-14,1.856180e-15,3.184437e-16
4,83,b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',...,b'TeV J1745-290',106,b'TeV J1745-2900',b'gal',0,3.341668e-11,2.440195e-12,1.828992e-13,5.610705e-15,6.128221e-17


Columns contained spectrum in gev:

In [4]:
gev_spectrum_columns = [
    'gev_nuFnu10000_100000',
    'gev_nuFnu1000_3000',
    'gev_nuFnu100_300',
    'gev_nuFnu3000_10000',
    'gev_nuFnu300_1000',
    'gev_nuFnu30_100',
]

Columns contained spectrum in simbad:

In [5]:
s_spectrum_columns = [
    's_FLUX_U', 
    's_FLUX_B', 
    's_FLUX_V', 
    's_FLUX_R', 
    's_FLUX_I',
    's_FLUX_G', 
    's_FLUX_J', 
    's_FLUX_H',
    's_FLUX_K', 
    's_FLUX_u',
    's_FLUX_g', 
    's_FLUX_r',
    's_FLUX_i',    
    's_FLUX_z',
    ]

Columns contained spectrum in tev:

In [6]:
tev_spectrum_columns =  [
    'tev_0.3TeV', 
    'tev_1TeV', 
    'tev_3TeV',  
    'tev_10TeV',  
    'tev_30TeV' 
]

In [7]:
spectrum_columns = gev_spectrum_columns + s_spectrum_columns + tev_spectrum_columns

In [8]:
spectrums = catalog[spectrum_columns + ['s_MAIN_ID'] + ['marked']].copy()

The next cell computes simbad spectrum using this formula:
$$ F_{simbad} = 10^{-flux_{simbad}}$$

In [9]:
for j in s_spectrum_columns:
    spectrums[j] = 10**(-spectrums[j])

The next cell computes TeV spectrum using:
$$F_{TeV} = 1.6*E^{2}*Flux$$

In [10]:
for i in tev_spectrum_columns:
    if len(re.findall("\d+",i)) == 2:
        E = 0.3
    else:
        E = int(re.findall("\d+",i)[0])
    spectrums[j] = 1.6 * E**2 * spectrums[j]

In [11]:
spectrums = spectrums.fillna(0)

In [12]:
display(spectrums)

,gev_nuFnu10000_100000,gev_nuFnu1000_3000,gev_nuFnu100_300,gev_nuFnu3000_10000,gev_nuFnu300_1000,gev_nuFnu30_100,s_FLUX_U,s_FLUX_B,s_FLUX_V,s_FLUX_R,...,s_FLUX_r,s_FLUX_i,s_FLUX_z,tev_0.3TeV,tev_1TeV,tev_3TeV,tev_10TeV,tev_30TeV,s_MAIN_ID,marked
0,2.938044e-12,7.446349e-11,4.520060e-12,3.103834e-11,7.171538e-11,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.0,3.341668e-11,2.440195e-12,1.828992e-13,5.610705e-15,6.128221e-17,b'SSTGC 519103',0
1,2.098420e-11,2.096862e-11,1.266728e-12,2.274123e-11,7.566604e-12,0.0,0.000000e+00,6.918312e-12,3.090294e-11,0.000000e+00,...,0.000000e+00,0.000000e+00,0.0,6.310589e-11,1.207092e-11,8.365091e-13,4.356098e-14,2.865129e-15,b'LS IV -05 11',0
2,2.938044e-12,7.446349e-11,4.520060e-12,3.103834e-11,7.171538e-11,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.0,3.341668e-11,2.440195e-12,1.828992e-13,5.610705e-15,6.128221e-17,b'GCIRS 1W',0
3,4.706814e-12,5.332763e-11,5.145968e-13,2.923599e-11,3.387356e-11,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.0,1.647138e-11,8.320850e-14,2.778103e-14,1.856180e-15,3.184437e-16,b'[MCD2010] 18',0
4,2.938044e-12,7.446349e-11,4.520060e-12,3.103834e-11,7.171538e-11,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.0,3.341668e-11,2.440195e-12,1.828992e-13,5.610705e-15,6.128221e-17,b'GCIRS 8',0
5,4.706814e-12,5.332763e-11,5.145968e-13,2.923599e-11,3.387356e-11,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'[MCD2010] 13',0
6,4.706814e-12,5.332763e-11,5.145968e-13,2.923599e-11,3.387356e-11,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.0,1.647138e-11,8.320850e-14,2.778103e-14,1.856180e-15,3.184437e-16,b'[MCD2010] 13',0
7,2.845686e-12,4.600289e-11,1.418847e-11,3.224835e-11,3.170300e-11,0.0,7.585778e-14,8.317639e-14,1.122019e-12,0.000000e+00,...,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'[MT91] 213',1
8,2.845686e-12,4.600289e-11,1.418847e-11,3.224835e-11,3.170300e-11,0.0,4.073800e-13,9.772367e-13,5.011870e-12,8.912505e-12,...,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'Schulte 21',0
9,5.929003e-12,8.840736e-11,1.353836e-10,4.055951e-11,1.317918e-10,0.0,5.370312e-12,2.454711e-12,1.778279e-11,6.456549e-11,...,0.000000e+00,0.000000e+00,0.0,3.033067e-11,5.807976e-13,5.926255e-14,1.818965e-15,2.823374e-16,b'LS I +61 303',1


Normilization:

In [13]:
spectrums[spectrum_columns] = spectrums[spectrum_columns].div(spectrums.sum(axis=1), axis=0)

The cell displays all information we have about spectrums now.

In [14]:
display(spectrums)

,gev_nuFnu10000_100000,gev_nuFnu1000_3000,gev_nuFnu100_300,gev_nuFnu3000_10000,gev_nuFnu300_1000,gev_nuFnu30_100,s_FLUX_U,s_FLUX_B,s_FLUX_V,s_FLUX_R,...,s_FLUX_r,s_FLUX_i,s_FLUX_z,tev_0.3TeV,tev_1TeV,tev_3TeV,tev_10TeV,tev_30TeV,s_MAIN_ID,marked
0,1.220356e-02,3.092940e-01,1.877467e-02,1.289218e-01,2.978794e-01,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.0,1.388006e-01,1.013568e-02,7.596960e-04,2.330481e-05,2.545438e-07,b'SSTGC 519103',0
1,1.313021e-02,1.312046e-02,7.926159e-04,1.422962e-02,4.734567e-03,0.0,0.000000e+00,4.328919e-03,1.933655e-02,0.000000e+00,...,0.000000e+00,0.000000e+00,0.0,3.948655e-02,7.553001e-03,5.234196e-04,2.725693e-05,1.792765e-06,b'LS IV -05 11',0
2,1.659270e-03,4.205349e-02,2.552718e-03,1.752900e-02,4.050149e-02,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.0,1.887218e-02,1.378108e-03,1.032929e-04,3.168664e-06,3.460932e-08,b'GCIRS 1W',0
3,1.105269e-02,1.252256e-01,1.208393e-03,6.865288e-02,7.954298e-02,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.0,3.867861e-02,1.953928e-04,6.523630e-05,4.358741e-06,7.477796e-07,b'[MCD2010] 18',0
4,1.160900e-02,2.942253e-01,1.785997e-02,1.226408e-01,2.833668e-01,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.0,1.320383e-01,9.641868e-03,7.226838e-04,2.216941e-05,2.421425e-07,b'GCIRS 8',0
5,5.245202e-03,5.942750e-02,5.734588e-04,3.258014e-02,3.774818e-02,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'[MCD2010] 13',0
6,5.150021e-03,5.834912e-02,5.630527e-04,3.198893e-02,3.706319e-02,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.0,1.802237e-02,9.104365e-05,3.039697e-05,2.030963e-06,3.484293e-07,b'[MCD2010] 13',0
7,2.845686e-12,4.600289e-11,1.418847e-11,3.224835e-11,3.170300e-11,0.0,7.585778e-14,8.317639e-14,1.122019e-12,0.000000e+00,...,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'[MT91] 213',1
8,7.519406e-04,1.215575e-02,3.749144e-03,8.521264e-03,8.377162e-03,0.0,1.076456e-04,2.582238e-04,1.324330e-03,2.355030e-03,...,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'Schulte 21',0
9,5.929002e-12,8.840736e-11,1.353836e-10,4.055951e-11,1.317918e-10,0.0,5.370312e-12,2.454711e-12,1.778279e-11,6.456548e-11,...,0.000000e+00,0.000000e+00,0.0,3.033067e-11,5.807976e-13,5.926255e-14,1.818965e-15,2.823374e-16,b'LS I +61 303',1
